# Phase 2: Model Training and Experimentation

This notebook will focus on training and optimizing our model. Our objectives include
1. Experiment tracking with MLflow
2. Cross-validation
3. Hyperparameter optimization
4. Model evaluation and selection
5. Model versioning and registration

In [7]:
# ignore this: required for run_notebooks.sh
%pip install mlflow optuna --quiet

/Users/nic/git/AmesHousingPredictor/.venv/bin/python: No module named pip
Note: you may need to restart the kernel to use updated packages.


In [8]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import joblib
import optuna
import xgboost as xgb

print("Loading data...")
import ames_notebooks
from app.data_ingestion.read_data import DataReader

reader = DataReader()
train_data, test_data = reader.load_train_test()
print("Train shape:", train_data.shape)
print("Test shape:", test_data.shape)

Loading data...
Train shape: (1460, 80)
Test shape: (1459, 79)


In [9]:
# get preprocessing pipeline
from app.pipelines.preprocessing import get_fitted_pipelines
feature_preprocessor, target_transformer = get_fitted_pipelines(train_data)

In [10]:
X = train_data.drop('SalePrice', axis=1)
y = train_data['SalePrice']

# apply pipelines/transformations
X_processed = feature_preprocessor.transform(X)
y_processed = target_transformer.transform(y)

# Split data into train and validation sets
# use _val to prevent confusion between the test dataset
X_train, X_val, y_train, y_val = train_test_split(
    X_processed, y_processed, 
    test_size=0.2, 
    random_state=42
)

# Apply preprocessing
print("Applying preprocessing pipeline...")
print("X_train shape:", X_train.shape)
print("X_val shape:", X_val.shape)

Applying preprocessing pipeline...
X_train shape: (1168, 241)
X_val shape: (292, 241)


## Baseline Model Development

Let's start with a simple baseline model using XGBoost with default parameters. This will give us a reference point for further improvements.

In [11]:
def evaluate_model(model, X, y, prefix=''):
    y_pred = model.predict(X)
    rmse = np.sqrt(mean_squared_error(y, y_pred))
    mae = mean_absolute_error(y, y_pred)
    r2 = r2_score(y, y_pred)
    
    return {
        f'{prefix}rmse': rmse,
        f'{prefix}mae': mae,
        f'{prefix}r2': r2
    }

In [20]:
# Configure MLflow
import mlflow
from app.config.settings import settings
# mlflow.set_tracking_uri("http://localhost:8000")
mlflow.set_tracking_uri("./mlruns")
mlflow.set_experiment(settings.MLFLOW_EXPERIMENT_NAME)

2025/11/12 10:50:05 INFO mlflow.tracking.fluent: Experiment with name 'ames-housing-pricing-experiment' does not exist. Creating a new experiment.


<Experiment: artifact_location='/Users/nic/git/AmesHousingPredictor/notebooks/mlruns/928459568205706713', creation_time=1762962605946, experiment_id='928459568205706713', last_update_time=1762962605946, lifecycle_stage='active', name='ames-housing-pricing-experiment', tags={}>

In [21]:
# for training baseline XGBoost model
with mlflow.start_run(run_name="xgboost-baseline"):
    # create model with default parameters
    model = xgb.XGBRegressor(
        random_state=42,
        n_jobs=-1
    )
    
    # train model
    model.fit(
        X_train, 
        y_train,
        eval_set=[(X_val, y_val)],
        verbose=100
    )
    
    train_metrics = evaluate_model(model, X_train, y_train, prefix='train_')
    val_metrics = evaluate_model(model, X_val, y_val, prefix='val_')
    
    mlflow.log_params(model.get_params())
    mlflow.log_metrics({**train_metrics, **val_metrics})
    mlflow.xgboost.log_model(model, "model")
    
    print("\nTraining Metrics:")
    for metric, value in train_metrics.items():
        print(f"{metric}: {value:.4f}")
    
    print("\nValidation Metrics:")
    for metric, value in val_metrics.items():
        print(f"{metric}: {value:.4f}")

# Save the baseline model
joblib.dump(model, '../models/baseline_xgboost.joblib')
print("\nBaseline model saved to ../models/baseline_xgboost.joblib")

[0]	validation_0-rmse:0.33887
[99]	validation_0-rmse:0.14973


2025/11/12 10:50:15 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
/Users/nic/git/AmesHousingPredictor/.venv/lib/python3.10/site-packages/xgboost/sklearn.py:1115: UserWarning: [10:50:15] WARNING: /Users/runner/work/xgboost/xgboost/src/c_api/c_api.cc:1575: Saving model in the UBJSON format as default.  You can use a file extension: `json` or `ubj` to choose between formats.
  self.get_booster().save_model(fname)
2025/11/12 10:50:18 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.
2025/11/12 10:50:18 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.



Training Metrics:
train_rmse: 0.0078
train_mae: 0.0054
train_r2: 0.9996

Validation Metrics:
val_rmse: 0.1497
val_mae: 0.1038
val_r2: 0.8799

Baseline model saved to ../models/baseline_xgboost.joblib


## Hyperparameter Optimization with Optuna

Now that we have a baseline model, let's use Optuna to find better hyperparameters for our XGBoost model. We'll define an objective function that Optuna will optimize using cross-validation scores.

In [22]:
# objective function for Optuna
def objective(trial):
    params = {
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.1, log=True),
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 7),
        'subsample': trial.suggest_float('subsample', 0.6, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
        'reg_alpha': trial.suggest_float('reg_alpha', 1e-8, 1.0, log=True),
        'reg_lambda': trial.suggest_float('reg_lambda', 1e-8, 1.0, log=True),
        'random_state': 42
    }
    
    # get model with suggested parameters
    model = xgb.XGBRegressor(**params)
    
    # cross-validation
    cv_scores = cross_val_score(
        model, 
        X_train, 
        y_train, 
        cv=5, 
        scoring='neg_root_mean_squared_error',
        n_jobs=-1
    )
    
    # mean negative RMSE (Optuna minimizes objective)
    return -cv_scores.mean()

# Create and run Optuna study
study = optuna.create_study(direction='minimize')
optuna.logging.set_verbosity(optuna.logging.WARNING)
study.optimize(objective, n_trials=50, show_progress_bar=True)

print("Best trial:")
trial = study.best_trial

print("  Value: ", trial.value)
print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

[I 2025-11-12 10:50:18,388] A new study created in memory with name: no-name-ea0e4606-a070-4f44-85f2-0daa33a66f0a


  0%|          | 0/50 [00:00<?, ?it/s]

Best trial:
  Value:  0.12477794471857348
  Params: 
    max_depth: 3
    learning_rate: 0.02709994303786845
    n_estimators: 814
    min_child_weight: 1
    subsample: 0.9300191782543125
    colsample_bytree: 0.648701250908197
    reg_alpha: 3.865348574052837e-08
    reg_lambda: 7.325087400964345e-05


In [ ]:
# for training the optimized model
with mlflow.start_run(run_name="xgboost-optimized"):
    # get model with best parameters
    best_params = study.best_params
    best_params['random_state'] = 42
    model = xgb.XGBRegressor(**best_params)
    
    model.fit(
        X_train, 
        y_train,
        eval_set=[(X_val, y_val)],
        verbose=100
    )

    
    train_metrics = evaluate_model(model, X_train, y_train, prefix='train_')
    val_metrics = evaluate_model(model, X_val, y_val, prefix='val_')
    
    mlflow.log_params(model.get_params())
    mlflow.log_metrics({**train_metrics, **val_metrics})
    mlflow.xgboost.log_model(model, name="model")
    
    print("\nTraining Metrics:")
    for metric, value in train_metrics.items():
        print(f"{metric}: {value:.4f}")
    
    print("\nValidation Metrics:")
    for metric, value in val_metrics.items():
        print(f"{metric}: {value:.4f}")

# save optimized model
joblib.dump(model, '../models/optimized_xgboost.joblib')
print("\nOptimized model saved to ../models/optimized_xgboost.joblib")

[0]	validation_0-rmse:0.42494
[100]	validation_0-rmse:0.16594
[200]	validation_0-rmse:0.14449
[300]	validation_0-rmse:0.13807
[400]	validation_0-rmse:0.13585
[500]	validation_0-rmse:0.13532
[600]	validation_0-rmse:0.13497
[700]	validation_0-rmse:0.13508
[800]	validation_0-rmse:0.13477
[813]	validation_0-rmse:0.13480


/Users/nic/git/AmesHousingPredictor/.venv/lib/python3.10/site-packages/xgboost/sklearn.py:1115: UserWarning: [10:50:52] WARNING: /Users/runner/work/xgboost/xgboost/src/c_api/c_api.cc:1575: Saving model in the UBJSON format as default.  You can use a file extension: `json` or `ubj` to choose between formats.
  self.get_booster().save_model(fname)
2025/11/12 10:50:54 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.
2025/11/12 10:50:54 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.



Training Metrics:
train_rmse: 0.0591
train_mae: 0.0451
train_r2: 0.9771

Validation Metrics:
val_rmse: 0.1348
val_mae: 0.0891
val_r2: 0.9026

Optimized model saved to ../models/optimized_xgboost.joblib


## Model Comparison

Let's compare the performance of our baseline and optimized models to see the improvement from hyperparameter optimization.

In [24]:
baseline_model = joblib.load('../models/baseline_xgboost.joblib')

# compare models on validation set
baseline_metrics = evaluate_model(baseline_model, X_val, y_val)
optimized_metrics = evaluate_model(model, X_val, y_val)

print("Baseline Model Metrics:")
for metric, value in baseline_metrics.items():
    print(f"{metric}: {value:.4f}")

print("\nOptimized Model Metrics:")
for metric, value in optimized_metrics.items():
    print(f"{metric}: {value:.4f}")

improvement = (baseline_metrics['rmse'] - optimized_metrics['rmse']) / baseline_metrics['rmse'] * 100
print(f"\nRMSE Improvement: {improvement:.2f}%")

Baseline Model Metrics:
rmse: 0.1497
mae: 0.1038
r2: 0.8799

Optimized Model Metrics:
rmse: 0.1348
mae: 0.0891
r2: 0.9026

RMSE Improvement: 9.97%


In [25]:
# Compose the preprocessor and trained model into a single sklearn-compatible object
# and use TransformedTargetRegressor so the target transformer is applied automatically.
from pathlib import Path
from sklearn.pipeline import Pipeline
from sklearn.compose import TransformedTargetRegressor

# Build an sklearn Pipeline that first applies the feature preprocessor then the model.
# Note: feature_preprocessor and model are expected to be already fitted objects.
skl_pipeline = Pipeline([
    ("preprocessor", feature_preprocessor),
    ("regressor", model),
])

# Wrap with TransformedTargetRegressor so predictions are returned on the original scale.
full_pipeline = TransformedTargetRegressor(regressor=skl_pipeline, transformer=target_transformer)
full_pipeline.fit(X, y)

# Save the composed object (sklearn pipeline or fallback wrapper)
models_path = Path("../models")
models_path.mkdir(parents=True, exist_ok=True)
joblib.dump(full_pipeline, models_path / "full_pipeline.joblib")
print(f"Saved full pipeline to {models_path / 'full_pipeline.joblib'}")

# Example: predict on a single row from the test set (keeps original columns)
example_row = test_data.iloc[[0]]  # DataFrame with one row
pred = full_pipeline.predict(example_row)
print("Example prediction (SalePrice):", pred)


Saved full pipeline to ../models/full_pipeline.joblib
Example prediction (SalePrice): [120046.805]
